In [1]:
import numpy as np
import pandas as pd
import seaborn
import scipy
import random
from scipy.stats import pearsonr
import matplotlib.pyplot as plot

c:\Users\yeohw\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# AES S-box
Sbox = [
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
]

In [3]:
# We set the number of traces to be used to be 100
no_of_traces = 100

# Code for calculating the hamming weight of the Sbox output
def hw(int_no):
    # Write Code to calculate the number of ones in a byte...
    c = 0
    while(int_no):
        int_no &= (int_no - 1)
        c += 1
    return c

In [4]:
# Prior to reading the CSV file, the CSV file is slightly editted
# 1) Add a row to the top
# 2) Remove any samples below index 100. This ensure that we work with only 100 samples

# Read the CSV file
df = pd.read_csv("waveform.csv")

df

,Plaintext,Ciphertext,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 2492,Unnamed: 2493,Unnamed: 2494,Unnamed: 2495,Unnamed: 2496,Unnamed: 2497,Unnamed: 2498,Unnamed: 2499,Unnamed: 2500,Unnamed: 2501
0,F22E10CA18045B15EC056215AF1E2FEE,6C4A1DE1148387B5645308323C59B7D6,0.478,0.456,0.416,0.436,0.398,0.380,0.422,0.482,...,0.402,0.362,0.358,0.404,0.400,0.370,0.398,0.360,0.352,0.416
1,00A51770A9A1269514B004D1A17489BF,BFE9B87BBB844AAE58765079F672CD61,0.460,0.470,0.444,0.464,0.430,0.386,0.422,0.460,...,0.406,0.374,0.348,0.408,0.400,0.364,0.400,0.370,0.350,0.394
2,7A3B778B2CE9AC3DFFBA500F84868DE7,8E6B3D1DF403BF01CEFCAD50DD34173C,0.438,0.446,0.414,0.450,0.402,0.372,0.416,0.456,...,0.410,0.380,0.354,0.396,0.398,0.368,0.410,0.378,0.354,0.396
3,463015576F33A2BCBC991B6D37C9AFBF,E1F96AB155DC977F7DA27E1E461E2613,0.484,0.458,0.430,0.434,0.410,0.380,0.434,0.458,...,0.394,0.356,0.358,0.394,0.392,0.376,0.384,0.372,0.342,0.430
4,0F0165AA74B57F97E8A11F802732AAF9,45208B01DF3A6D9E42E0156BFC08EC24,0.472,0.470,0.444,0.456,0.406,0.396,0.426,0.450,...,0.404,0.380,0.344,0.392,0.390,0.372,0.414,0.368,0.346,0.396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,034177FD2F54E8B44D2928209F68CC3F,9F5C7CF2753E403CC7876511163A7C9C,0.442,0.430,0.400,0.432,0.392,0.366,0.408,0.460,...,0.406,0.350,0.362,0.392,0.392,0.344,0.394,0.366,0.342,0.406
96,876B10FCA9BCD6EB7C8D8F8B8DE469E2,67A99EACFB8796DE11082EC35C80BC4B,0.494,0.464,0.436,0.450,0.428,0.384,0.430,0.472,...,0.392,0.366,0.354,0.404,0.386,0.366,0.388,0.372,0.346,0.422
97,0B96A9FB2324C321ABF2F5F67B600584,947EF54A3D15A33A7FB35F1C68417B93,0.440,0.464,0.440,0.458,0.428,0.390,0.416,0.458,...,0.406,0.364,0.346,0.400,0.384,0.374,0.392,0.362,0.346,0.372
98,4A1A8BA61372C2B3DDB3BDB866EF438F,79FEC2829DEDF9D5BFDACB5E6A323367,0.476,0.454,0.428,0.436,0.416,0.378,0.426,0.486,...,0.386,0.362,0.356,0.394,0.384,0.366,0.392,0.358,0.338,0.398


In [5]:
# Extract the contents of the Plaintext and append them to an array
data_arr = df["Plaintext"].to_numpy()

len(data_arr)
data_arr[0]

'F22E10CA18045B15EC056215AF1E2FEE'

In [25]:
# Extract the 16 bytes of the plaintext and store them into a separate array
plaintext_bytes = [] # Matrix to store plaintext information
byte_array = []

base = 0

for i in range(0, 16):
    for j in range(0, len(data_arr)):
        byte_array.append(int(data_arr[j][base : (base + 2)], 16))
    plaintext_bytes.append(byte_array)  
    base = base + 2
    byte_array = []

#plaintext_bytes[1]

In [28]:
leaky_sbox_output_value_array = []
hamming_weight_of_leaky_sbox_bytes = []

no_of_possible_values_of_key_byte = 256
power_model_matrix = [[]]*no_of_possible_values_of_key_byte

power_model_matrix_16_bytes = []


In [ ]:
# Calculate power model matrix for all 16 plaintext bytes
for i in range(0, 16):
    for key_byte_guess in range(0,no_of_possible_values_of_key_byte):
        k = key_byte_guess

        leaky_sbox_output_value_array = []
        for byte_pos in range(0,no_of_traces):
            byte_now = plaintext_bytes[i][byte_pos] ^ k
            Sbox_output_leaky_value = Sbox[byte_now]
            leaky_sbox_output_value_array.append(Sbox_output_leaky_value)

        hamming_weight_of_leaky_sbox_bytes = []
        for byte in range(0,no_of_traces):
            hamming_weight_of_leaky_sbox_bytes.append(hw(leaky_sbox_output_value_array[byte]))

        power_model_matrix[key_byte_guess] = hamming_weight_of_leaky_sbox_bytes
    power_model_matrix_16_bytes.append(power_model_matrix)
    power_model_matrix = [[]]*no_of_possible_values_of_key_byte

In [30]:
power_model_matrix_16_bytes[0]

[[3,
  4,
  5,
  4,
  5,
  6,
  3,
  6,
  0,
  2,
  4,
  5,
  3,
  5,
  5,
  4,
  2,
  2,
  4,
  4,
  1,
  5,
  4,
  7,
  3,
  5,
  4,
  3,
  6,
  4,
  5,
  6,
  5,
  6,
  5,
  2,
  4,
  4,
  4,
  6,
  3,
  7,
  4,
  5,
  4,
  3,
  6,
  2,
  6,
  5,
  4,
  4,
  2,
  6,
  6,
  5,
  4,
  5,
  1,
  4,
  4,
  6,
  3,
  5,
  8,
  5,
  3,
  6,
  3,
  5,
  5,
  5,
  4,
  6,
  3,
  5,
  4,
  2,
  6,
  5,
  4,
  2,
  2,
  5,
  2,
  4,
  4,
  5,
  3,
  3,
  3,
  7,
  6,
  4,
  3,
  6,
  4,
  4,
  5,
  4],
 [3,
  5,
  2,
  2,
  5,
  4,
  5,
  4,
  6,
  2,
  3,
  4,
  6,
  3,
  2,
  4,
  4,
  2,
  7,
  3,
  7,
  3,
  2,
  1,
  3,
  1,
  5,
  4,
  3,
  3,
  4,
  4,
  4,
  4,
  5,
  2,
  3,
  6,
  7,
  3,
  4,
  1,
  3,
  3,
  2,
  2,
  3,
  3,
  3,
  4,
  6,
  3,
  4,
  3,
  3,
  5,
  2,
  5,
  7,
  2,
  4,
  3,
  6,
  4,
  1,
  5,
  4,
  7,
  5,
  4,
  5,
  4,
  6,
  3,
  4,
  5,
  2,
  3,
  3,
  6,
  6,
  5,
  4,
  4,
  5,
  5,
  6,
  5,
  2,
  5,
  4,
  1,
  6,
  2,
  4,
  6,
  2,
  5,
  5,
  3]

In [31]:
correlation_matrix_16_bytes = []
correlation_matrix = []
correlation_values = []
power_trace_len = 2500

for i in range(0, 16):
    for power_trace_sample in range(0, power_trace_len):
        for key_byte_guess in range(0,no_of_possible_values_of_key_byte):
            model_trace = power_model_matrix_16_bytes[i][key_byte_guess]
            corr_value = scipy.stats.pearsonr(df["Unnamed: {x}".format(x=str(2 + power_trace_sample))].to_numpy(), model_trace)
            correlation_values.append(corr_value[0])
        
        correlation_matrix.append(correlation_values)
        correlation_values = []
    correlation_matrix_16_bytes.append(correlation_matrix)
    correlation_matrix = []

KeyboardInterrupt: 